<div style="display: flex; justify-content: space-between; align-items: center; padding: 10 px;">
    <h2 style="margin: 0 0 20px; color: #FFFFFF; line-height: 1">CS5002: Programming Principles and Practice</h2>
    <img src="../rsc/logo.png" alt="University Logo" style="width: 80px; height: auto;"/>
</div>
                                  <!--  -->
<div style="display: flex; justify-content: space-between; width: 100%;">
  <span style="text-align: left;">Practical P3: Data analysis and visualisation Python</span>
  <span style="text-align: right; font-size: 12px;">Student ID: 200013825</span>
</div> 


#### Introduction 
This project aims to modularly read, clean and analyze real-world csv data. Below is the introduction on all the modules and major functions that worked together to complete data analysis.

`data_mining.py`: This executable python 
script automates data cleaning process, I used 
```python
invalid_mask = ~data[column].isin(valid_set)
```
to collect a list of booleans, indicating whether each csv code is admissible. 

Based on the context of this dataset, it is more efficient to neglect the type of the data when checking consistency, and then convert all data into pandas' categorical type. 
``` python
data[column] = data[column].astype(str)
```
I have converted all column in the data as string to help the determination of admissibili

`data_analysis.py`



#### Problem Encountered

##### __1. infinite commit loop__: 
When I was trying to set up hooks for git to automatically update the gitlog everytime after commit, i accidently created an infinite loop of git commit, below was what I have included in the post-commit* file in .git/hooks:

``` python
#!/bin/bash
git log --oneline  --graph  > gitlog.txt
git commit -m 'gitlog.txt is updated with the latest commit history'
```
and this created redundant git log histroy that are shown in the gitlog.txt

I fixed it by deleting the line to commit within post-commit, after i realized that adding commit within post-commit can trigger recursive calls.

In [ ]:
import json
import pandas as pd

We start with exploring the content of the dataset.

In [ ]:
df = pd.read_csv("data/Scotland_teaching_file_1PCT.csv")

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 63388 entries, 0 to 63387
Data columns (total 17 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   Record_Number             63388 non-null  int64 
 1   Region                    63388 non-null  object
 2   RESIDENCE_TYPE            63388 non-null  object
 3   Family_Composition        63388 non-null  object
 4   sex                       63388 non-null  int64 
 5   age                       63388 non-null  int64 
 6   Marital_Status            63388 non-null  int64 
 7   student                   63388 non-null  int64 
 8   Country_Of_Birth          63388 non-null  int64 
 9   health                    63388 non-null  int64 
 10  Ethnic_Group              63388 non-null  int64 
 11  religion                  63388 non-null  int64 
 12  Economic_Activity         63388 non-null  object
 13  Occupation                63388 non-null  object
 14  industry              

In [ ]:
for column, dtype in df.dtypes.items():
    print(f"{column}: {dtype}")

Record_Number: int64
Region: object
RESIDENCE_TYPE: object
Family_Composition: object
sex: int64
age: int64
Marital_Status: int64
student: int64
Country_Of_Birth: int64
health: int64
Ethnic_Group: int64
religion: int64
Economic_Activity: object
Occupation: object
industry: object
Hours_Worked_Per_Week: object
Approximate_Social_Grade: object


Next, we read the JSON file with the interpretations of categories. And print all the types of the JSON dictionary

In [ ]:
with open("./data/data_dictionary.json", "r") as read_file:
    labels = json.load(read_file)

for i in labels:
    print(type(i))

<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
